In [ ]:
import matplotlib.animation
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.rcParams["figure.autolayout"] = True
plt.rcParams['figure.constrained_layout.use'] = True
plt.ioff()
%matplotlib inline

## Schematic
![IMAGE](../data/Strandbeest_Leg_Proportions.png)

## Utilities for analysis 

In [ ]:
def four_bar_pos_solver(a, b, c, d, 
                        ta, tb, mode=0):
    
    x = a*np.cos(ta) + b*np.cos(tb)
    y = a*np.sin(ta) + b*np.sin(tb)
    
    R = np.sqrt(x**2 + y**2)
        
    Ac = (c**2 - d**2 + R**2)/(2*c)
    Ad = (d**2 - c**2 + R**2)/(2*d)
    
    tc = 2*np.arctan2(- y + ((-1)**mode)*np.sqrt(R**2 - Ac**2), -x-Ac)
    td = 2*np.arctan2(- y - ((-1)**mode)*np.sqrt(R**2 - Ad**2), -x-Ad)
    
    return tc, td

def four_bar_vel_solver(a, b, c, d, 
                        ta, tb, tc, td,
                        wa, wb):
    
    A = np.array([[c*np.cos(tc), d*np.cos(td)],
                  [c*np.sin(tc), d*np.sin(td)]])
    
    y = np.array([a*wa*np.cos(ta)+b*wb*np.cos(tb),
                  a*wa*np.sin(ta)+b*wb*np.sin(tb)])
    
    wc, wd = np.linalg.solve(A,y.T)
    return wc, wd

def four_bar_acc_solver(a, b, c, d,
                        ta, tb, tc, td,
                        wa, wb, wc, wd,
                        aa, ab):
    
    A = np.array([[c*np.cos(tc), d*np.cos(td)],
                  [c*np.sin(tc), d*np.sin(td)]])

    y = np.array([( a*aa*np.cos(ta) - a*(wa**2)*np.sin(ta) + c*(wc**2)*np.sin(tc)
                   +b*ab*np.cos(tb) - b*(wb**2)*np.sin(tb) + d*(wd**2)*np.sin(td)),
                  ( a*aa*np.sin(ta) + a*(wa**2)*np.cos(ta) - c*(wc**2)*np.cos(tc)
                   +b*ab*np.sin(tb) + b*(wb**2)*np.cos(tb) - d*(wd**2)*np.cos(td))])
    
    ac, ad = np.linalg.solve(A, y.T)
    return ac, ad

def four_bar_solver(a, b, c, d,
                    ta, tb,
                    wa, wb,
                    aa, ab,
                    mode=0):

    tc, td = four_bar_pos_solver(a, b, c, d, 
                                 ta, tb, mode=mode)
    
    wc, wd = four_bar_vel_solver(a, b, c, d, 
                                 ta, tb, tc, td,
                                 wa, wb)
    
    ac, ad = four_bar_acc_solver(a, b, c, d,
                                 ta, tb, tc, td,
                                 wa, wb, wc, wd,
                                 aa, ab)

    return tc, td, wc, wd, ac, ad

def four_bar_plot(a, b, c, d, 
                  ta, tb, tc, td, 
                  ox=0, oy=0):
    
    plt.xlim([-80,60])
    plt.ylim([-100,60])
    plt.gca().set_aspect('equal')

    plt.grid("on")
    plt.plot([ox+0, ox+a*np.cos(ta)], 
             [oy+0, oy+a*np.sin(ta)])

    plt.plot([ox+a*np.cos(ta), ox+a*np.cos(ta)+b*np.cos(tb)], 
             [oy+a*np.sin(ta), oy+a*np.sin(ta)+b*np.sin(tb)])

    plt.plot([ox+0, ox+d*np.cos(td)], 
             [oy+0, oy+d*np.sin(td)])

    plt.plot([ox+d*np.cos(td), ox+d*np.cos(td)+c*np.cos(tc)], 
             [oy+d*np.sin(td), oy+d*np.sin(td)+c*np.sin(tc)])    

In [ ]:
class Point():
    def __init__(self, a=0, ta=0, wa=0, aa=0):

        self.pos = np.array([a*np.cos(ta), a*np.sin(ta)])
        self.vel = np.array([-a*wa*np.sin(ta), a*wa*np.cos(ta)])
        self.acc = np.array([-a*(wa**2)*np.cos(ta) - a*aa*np.sin(ta),
                             -a*(wa**2)*np.sin(ta) + a*(aa)*np.cos(ta)])

    def __add__(self, b):
        c = Point()
        c.pos = self.pos + b.pos
        c.vel = self.vel + b.vel
        c.acc = self.acc + b.acc
        return c
        
    def __sub__(self, b):
        c = Point()
        c.pos = self.pos - b.pos
        c.vel = self.vel - b.vel
        c.acc = self.acc - b.acc
        return c

    def __neg__(self):
        c = Point()
        c.pos = - self.pos
        c.vel = - self.vel
        c.acc = - self.acc
        return c

    def __repr__(self):
        str = f"__name__\n"
        str += f"Position    : {self.pos}\n"
        str += f"Velocity    : {self.vel}\n"
        str += f"Acceleration: {self.acc}\n"
        return str



## Jansen Linkage Kinematic Analysis

In [ ]:
class jansenLinkage():
    
    def __init__(self):
        
        self.link = {"b":41.5, "c":39.3,
                     "d":40.1, "e":55.8, "f":39.4,
                     "g":36.7, "h":65.7, "i":49.0,
                     "j":50.0, "k":61.9, "n":(38.0**2 + 7.8**2)**0.5,
                     "m":15.0}
        
        self.tn = np.arctan2(7.8, 38.0)
    
    
    def kinematic_analysis(self, tm, wm=0, wn=0, am=0, an=0):

        tn = self.tn
        
        ## eqn 1
        tj, tb, wj, wb, aj, ab = four_bar_solver(self.link["n"], self.link["m"],
                                                 self.link["j"], self.link["b"],
                                                 self.tn, tm,
                                                 wn, wm,
                                                 an, am,
                                                 mode=0)
        ## eqn 2
        tk, tc, wk, wc, ak, ac = four_bar_solver(self.link["n"], self.link["m"],
                                                 self.link["k"], self.link["c"],
                                                 self.tn, tm,
                                                 wn, wm,
                                                 an, am,
                                                 mode=1)
        ## eqn 4
        td = tb + np.arccos((self.link["b"]**2+self.link["d"]**2-self.link["e"]**2)/(2*self.link["b"]*self.link["d"])) + np.pi
        wd = wb
        ad = ab
        
        ## eqn 3
        tg, tf, wg, wf, ag, af = four_bar_solver(self.link["d"], self.link["c"],
                                                 self.link["g"], self.link["f"],
                                                 td, tc,
                                                 wd, wc,
                                                 ad, ac,
                                                 mode=1)
        ## eqn 5
        ti = tg + np.arccos((self.link["g"]**2+self.link["i"]**2-self.link["h"]**2)/(2*self.link["g"]*self.link["i"])) - np.pi
        wi = wg
        wh = wg
        ai = ag
        ah = ag
        
        ## eqn 6
        px = self.link["c"]*np.cos(tc) + self.link["i"]*np.cos(ti)
        py = self.link["c"]*np.sin(tc) + self.link["i"]*np.sin(ti)

        pos = { "tb":tb, "tc":tc, "td":td,
                "tf":tf, "tg":tg, "ti":ti,
                "tj":tj, "tk":tk, "tn":self.tn,
                "tm":tm, "px":px, "py":py}
        
        ang_vel = { "wb":wb, "wc":wc, "wd":wd,
                "wf":wf, "wg":wg, "wi":wi,
                "wj":wj, "wk":wk, "wn":wn,
                "wm":wm}
        
        ang_acc = { "ab":ab, "ac":ac, "ad":ad,
                "af":af, "ag":ag, "ai":ai,
                "aj":aj, "ak":ak, "an":an,
                "am":am}

        # declare points for kinematic analysis
        vecs = {}
        for ci in ["b", "c", "d", "f", "g", "i", "j", "k", "n", "m"]:
            vecs[ci] = Point(self.link[ci], pos[f"t{ci}"], ang_vel[f"w{ci}"], ang_acc[f"a{ci}"])

        points = {"p0":Point(0,0,0,0),      "p3":vecs["n"],
                  "p1":vecs["n"]+vecs["m"], "p2":vecs["b"],
                  "p7":vecs["c"]+vecs["i"], "p5":vecs["c"],
                  "p6":vecs["c"]-vecs["g"], "p4":-vecs["d"]}

        return pos, ang_vel, ang_acc, points
        
    def plot_jansen_linkage(self, idx):
        plt.cla()
        res, _, _, _ = self.kinematic_analysis(2*idx*np.pi/180)

        four_bar_plot(self.link["n"], self.link["m"],
                      self.link["j"], self.link["b"],
                      res["tn"], res["tm"], res["tj"], res["tb"])
        
        four_bar_plot(self.link["n"], self.link["m"],
                      self.link["k"], self.link["c"],
                      res["tn"], res["tm"], res["tk"], res["tc"])
        
        four_bar_plot(self.link["d"], self.link["c"],
                      self.link["g"], self.link["f"],
                      res["td"], res["tc"], res["tg"], res["tf"],
                      ox = -(self.link["d"]*np.cos(res["td"])),
                      oy = -(self.link["d"]*np.sin(res["td"])))
        
        plt.plot([self.link["c"]*np.cos(res["tc"]),res["px"]],
                 [self.link["c"]*np.sin(res["tc"]),res["py"]])

        plt.plot([self.link["c"]*np.cos(res["tc"])-self.link["g"]*np.cos(res["tg"]),res["px"]],
                 [self.link["c"]*np.sin(res["tc"])-self.link["g"]*np.sin(res["tg"]),res["py"]])
        
        plt.plot([self.link["b"]*np.cos(res["tb"]), -self.link["d"]*np.cos(res["td"])],
                 [self.link["b"]*np.sin(res["tb"]), -self.link["d"]*np.sin(res["td"])])
        
        plt.plot(res["px"], res["py"], "ro")
        
        
        
    def display(self):
        
        self.fig, self.ax = plt.subplots(figsize=(10,5))
        self.animation = matplotlib.animation.FuncAnimation(self.fig, 
                                           self.plot_jansen_linkage, 
                                           frames=180)
        
        writer = matplotlib.animation.PillowWriter(fps=72)
        self.animation.save("../outputs/jansen_linkage.gif", writer=writer)
        

## Animation

In [ ]:
jl = jansenLinkage()
jl.display()
jl.animation

## Position, Velocity, Acceleration Plots

In [ ]:
jl = jansenLinkage()

pos = {}
vel = {}
acc = {}

for i in range(8):
    pos[f"p{i}x"] = []
    pos[f"p{i}y"] = []
    vel[f"v{i}x"] = []
    vel[f"v{i}y"] = []
    acc[f"a{i}x"] = []
    acc[f"a{i}y"] = []

for i in range(360):
    
    tm = i*np.pi/180
    wm = 1
    wn = 0
    am = 0
    an = 0
    
    _, _, _, points = jl.kinematic_analysis(tm, wm, wn, am, an)
    for i in range(8):
        pos[f"p{i}x"].append(points[f"p{i}"].pos[0])
        pos[f"p{i}y"].append(points[f"p{i}"].pos[1])
        vel[f"v{i}x"].append(points[f"p{i}"].vel[0])
        vel[f"v{i}y"].append(points[f"p{i}"].vel[1])
        acc[f"a{i}x"].append(points[f"p{i}"].acc[0])
        acc[f"a{i}y"].append(points[f"p{i}"].acc[1])

# Position plots (x vs y)
f, axs = plt.subplots(4, 2, figsize=(8, 6))

for i in range(4):
    plt.subplot(4,2,2*i+1)
    ax = plt.gca()
    #ax.set_aspect('equal', adjustable='box')
    plt.title(f"p{2*i}")
    plt.grid("on")
    plt.plot(pos[f"p{2*i}x"], pos[f"p{2*i}y"], "orange")
    
    plt.subplot(4,2,2*i+2)
    ax = plt.gca()
    #ax.set_aspect('equal', adjustable='box')
    plt.title(f"p{2*i+1}")
    plt.grid("on")
    plt.plot(pos[f"p{2*i+1}x"], pos[f"p{2*i+1}y"], "orange")

plt.savefig("../outputs/kinematic_analysis/position_plots.png")

# Position plots
f, axs = plt.subplots(8, 2, figsize=(8, 12))

for i in range(8):
    plt.subplot(8,2,2*i+1)
    plt.title(f"p{i}x")
    plt.grid("on")
    #plt.ylim([-45, 20])
    plt.xlim([0, 360])
    plt.plot(pos[f"p{i}x"], "r")
    
    plt.subplot(8,2,2*i+2)
    plt.title(f"p{i}y")
    plt.grid("on")
    #plt.ylim([-45, 20])
    plt.xlim([0, 360])
    plt.plot(pos[f"p{i}y"], "r")

plt.savefig("../outputs/kinematic_analysis/position_angle_plots.png")

# Velocity plots
f, axs = plt.subplots(8, 2, figsize=(8, 12))

for i in range(8):
    plt.subplot(8,2,2*i+1)
    plt.title(f"v{i}x")
    plt.grid("on")
    #plt.ylim([-45, 20])
    plt.xlim([0, 360])
    plt.plot(vel[f"v{i}x"], "g")
    
    plt.subplot(8,2,2*i+2)
    plt.title(f"v{i}y")
    plt.grid("on")
    #plt.ylim([-45, 20])
    plt.xlim([0, 360])
    plt.plot(vel[f"v{i}y"], "g")

plt.savefig("../outputs/kinematic_analysis/velocity_angle_plots.png")

# Acceleration plots
f, axs = plt.subplots(8, 2, figsize=(8, 12))

for i in range(8):
    plt.subplot(8,2,2*i+1)
    plt.title(f"a{i}x")
    plt.grid("on")
    #plt.ylim([-45, 20])
    plt.xlim([0, 360])
    plt.plot(acc[f"a{i}x"], "b")
    
    plt.subplot(8,2,2*i+2)
    plt.title(f"a{i}y")
    plt.grid("on")
    #plt.ylim([-45, 20])
    plt.xlim([0, 360])
    plt.plot(acc[f"a{i}y"], "b")
    
plt.savefig("../outputs/kinematic_analysis/acceleration_angle_plots.png")


In [ ]:
_,_,_, points = jl.kinematic_analysis(0, 1)
print(points)

## Testing

In [ ]:
a = 1
b = 1
c = 1
d = 1.1

ta = 0
tb = np.pi+1
wa = 0
wb = 1
aa = 0
ab = 0

tc, td = four_bar_pos_solver(a, b, c, d, 
                             ta, tb, mode=1)

wc, wd = four_bar_vel_solver(a, b, c, d,
                             ta, tb, tc, td,
                             wa, wb)

ac, ad = four_bar_acc_solver(a, b, c, d,
                             ta, tb, tc, td,
                             wa, wb, wc, wd,
                             aa, ab)

print(aa, ab, ac, ad)

four_bar_plot(a, b, c, d,
             ta, tb, tc, td,
             ox = 0, oy = 0)
plt.xlim([-2, 2])
plt.ylim([-2, 2])
